# word2vec averaged repr. of a sentence

## training word2vec on rock corpus

In [82]:
import gensim,os
import numpy as np
from scipy.spatial.distance import cosine
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()

train_path='rank/train/'
sentences = MySentences(train_path) 
model = gensim.models.Word2Vec(sentences,size=100, window=5, min_count=5, workers=4)

In [9]:
import numpy as np
from scipy.spatial.distance import cosine

In [ ]:
dfdf11

In [145]:
v1=make_feature_vec('I love you',model,100)
v2=make_feature_vec('I love you baby',model,100)
v3=make_feature_vec('moon is shining',model,100)

In [146]:
print cosine(v1,v2)
print cosine(v1,v3)
print cosine(v2,v3)

0.0280046085098
0.99573738118
0.988993420945


In [144]:
def clean_word(word): 
    return re.sub(u'[^A-Za-z]','',word)

def make_feature_vec(words, model, num_features):
    # average all of the word vectors in a sentence
    #words=nlp(words)
    #words=[token.lemma_ for token in words]
    words=words.split(' ')
    #print (words)
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0
    for word in words:
        word = clean_word(word)
        if word not in model:
            #print(word)
            nlp=spacy.load('en')
            word = nlp(word)
            try:
                word = [i.lemma_ for i in word][0]
            except IndexError:
                continue

        if word not in model:
            continue

        featureVec = np.add(featureVec,model[word])
        nwords = nwords + 1
    featureVec = np.divide(featureVec,nwords)
    return featureVec

## save, load model

In [139]:
fname='rock_train.w2v'
#model.save(fname)
model = Word2Vec.load(fname)  # you can continue training with the loaded model!

# D2V

In [32]:
# gensim modules: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
import smart_open


def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

def remove_empty_lines(fn):
    f=open(fn,'r').read().split('\n')
    output=fn.replace('.txt','_noemp.txt')
    open(output,'w').close
    g=open(output,'a')
    for line in f:
        if line!='':
            g.write(line+'\n')

## train

In [31]:
#clean file to remove empty lines if necessary
train_file='rank/train/lyrics_train_data_clean.txt'
test_file='rank/test/lyrics_test_data_clean.txt'
remove_empty_lines(train_file)
remove_empty_lines(test_file)

In [33]:
#preprocess corpus into gensim format
train_file='rank/train/lyrics_train_data_clean_noemp.txt'
test_file='rank/test/lyrics_test_data_clean_noemp.txt'
train_corpus = list(read_corpus(train_file))
test_corpus = list(read_corpus(test_file, tokens_only=True))

In [34]:
train_corpus[:2]

[TaggedDocument(words=[u'may', u'seem', u'unusual', u'but', u'there', u'is', u'really', u'nothing', u'wrong', u'with', u'me'], tags=[0]),
 TaggedDocument(words=[u'tell', u'you', u'my', u'secrets', u'but', u'you', u'think', u'that', u'boring'], tags=[1])]

In [39]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2)
model.build_vocab(train_corpus)

In [40]:
for epoch in range(10):
    print epoch
    model.train(train_corpus, total_examples=model.corpus_count)
    model.alpha -= 0.002  
    model.min_alpha = model.alpha  

0
1
2
3
4
5
6
7
8
9


In [ ]:
#alternative training
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2,iter=15)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count)

In [136]:
fname='rock_train.d2v'
#model.save(fname)
model = Doc2Vec.load(fname)

## inference

In [44]:
x='Oh never let you go'
y="I'll never let you go"
z="It's my turn, searching for another break"



a=model.infer_vector(x.split(' '))
b=model.infer_vector(y.split(' '))
c=model.infer_vector(z.split(' '))
print cosine(a,b)
print cosine(a,c)
print cosine(b,c)

0.296794090269
0.952921933758
0.901668460463


In [67]:
test_pas="""I wanna love you like it ain't no secret
Like I'm not ashamed to show
Nor would I ever, oh never never
Oh never let you go
I'll never let you go"""

t=test_pas.split('\n')
e=model.infer_vector(t[0].split(' '))
a=model.infer_vector(t[1].split(' '))
b=model.infer_vector(t[2].split(' '))
c=model.infer_vector(t[3].split(' '))
d=model.infer_vector(t[4].split(' '))
print cosine(e,a)
print cosine(a,b)
print cosine(b,c)
print cosine(c,d)


0.77862892319
1.00376286212
0.357445432395
0.542003353344


In [134]:
test_pas="""I wanna love you like it ain't no secret
Like I'm not ashamed to show
Nor would I ever, oh never never
Oh never let you go
I'll never let you go"""

t=test_pas.split('\n')
vec=[]
for line in t:
    l=line.split(' ')
    vec.append(l)
print vec

[['I', 'wanna', 'love', 'you', 'like', 'it', "ain't", 'no', 'secret'], ['Like', "I'm", 'not', 'ashamed', 'to', 'show'], ['Nor', 'would', 'I', 'ever,', 'oh', 'never', 'never'], ['Oh', 'never', 'let', 'you', 'go'], ["I'll", 'never', 'let', 'you', 'go']]


In [138]:
b=model.infer_vector(vec[0])

In [115]:
model = gensim.models.KeyedVectors.load_word2vec_format('model/GoogleNews-vectors-negative300.bin', binary=True)  


In [75]:
def make_feature_vec(words, model, num_features):
    # average all of the word vectors in a sentence
    words=words.split(' ')
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    for word in words:
        nwords = nwords + 1.
        featureVec = np.add(featureVec,model[word])
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [122]:
v1=make_feature_vec('I love you',model,300)


In [132]:
xx = np.nan_to_num(v1)


In [118]:
a=[[-1.9321557680765789, 0.77957312279103508, 0.50855190776400561, 0.34873870429582832, 0, 0, 40, 28, 1], [-3.41674314226423, 0.48945975253308649, 0.59788698690639319, 0.35616330825118925, 0, 1, 28, 32, 1], [-1.721189308166504, 0.52949953423100071, 0.26509378411068274, 0.19793695619404383, 0, 0, 32, 19, 1], [-1.1092583656311035, 0.26562739714544592, 0.10523526352690327, 0.070908816034415123, 1, 0, 19, 21, 1]]

In [119]:
for i in a: print i

[-1.9321557680765789, 0.7795731227910351, 0.5085519077640056, 0.3487387042958283, 0, 0, 40, 28, 1]
[-3.41674314226423, 0.4894597525330865, 0.5978869869063932, 0.35616330825118925, 0, 1, 28, 32, 1]
[-1.721189308166504, 0.5294995342310007, 0.26509378411068274, 0.19793695619404383, 0, 0, 32, 19, 1]
[-1.1092583656311035, 0.2656273971454459, 0.10523526352690327, 0.07090881603441512, 1, 0, 19, 21, 1]


In [120]:
b=[[-2.9132102966308593, 0.84677078160842412, 0.77228926608025872, 0.73800714765249742, 0, 0, 25, 20, 1], [-2.3162093692355685, 0.85949340103096816, 0.80950776360602306, 0.69511529949504136, 0, 0, 20, 50, 1], [-2.026075839996338, 0.79122691193538486, 0.69034124896942539, 0.576113041466773, 0, 0, 50, 18, 1], [-1.7705413273402624, 1.1136524156219227, 0.57432313199626739, 0.44441332938962597, 0, 0, 18, 65, 1], [-3.4620275497436523, 0.88504072570899339, 0.86365218545666289, 0.64107502369887337, 0, 0, 65, 21, 1]]

In [121]:
for i in b: print i

[-2.9132102966308593, 0.8467707816084241, 0.7722892660802587, 0.7380071476524974, 0, 0, 25, 20, 1]
[-2.3162093692355685, 0.8594934010309682, 0.8095077636060231, 0.6951152994950414, 0, 0, 20, 50, 1]
[-2.026075839996338, 0.7912269119353849, 0.6903412489694254, 0.576113041466773, 0, 0, 50, 18, 1]
[-1.7705413273402624, 1.1136524156219227, 0.5743231319962674, 0.44441332938962597, 0, 0, 18, 65, 1]
[-3.4620275497436523, 0.8850407257089934, 0.8636521854566629, 0.6410750236988734, 0, 0, 65, 21, 1]


# right now it seems like LM is behaving not the best, but w2v, d2v, and googlew2v behave all kind of reasonable as a sanity check. we can proceed.

<h2 style="background:blue;color:white;padding:2pt;"> if you have time </h2>

## detecting gibberish

to do better on detecting gibberish sentence here is an idea:

1. build a data set contains 10000 randomly sampled sentences from lyrics;(real)

2. generate 20,000 sents from the LM, rank them by loglik, then take the bottom half 10,000 to add to the data set;

3. train this data set on d2v;

4. train a doc classifier that classifies lyrics frmo gibberish lyrics;

5. for ranking, for each candidate line generate a prediction of whether this is gibberish or not and add it to the feature set. when training the next line classifier, you can also compute this feature of course (for pos examples they should always be 1)

## LM

now the smoothing (KN) will cause "I am zangsir" to have much lower likelihood than "I am John". Converting all proper names to a tag would help. But again, this would not happen within the system. And that'll have to be preprocessed before the training of LM.

# other text discourse coherence features:
 - vocabulary introduction: count the new vocab in consecutive blockes of texts
 - lexical chains: looks at a sliding window of text and see the lexical overlaps 

https://corpling.uis.georgetown.edu/compdisc/webtile.html

# Rhyming


## generic rhyming function

In [46]:
import pronouncing
def is_rhyme(word1,word2):
    return word1 in pronouncing.rhymes(word2)
print is_rhyme('hood?','good')
print is_rhyme('hood','hooded')

False
False


In [61]:
def is_rhyme(word1,word2):
    word1=re.sub(u'[^A-Za-z]','',word1)
    word2=re.sub(u'[^A-Za-z]','',word2)
    return int((word1 in pronouncing.rhymes(word2)) or word1==word2)

In [64]:
import re
print is_rhyme('go','show')

1


## sent internal rhyming

for a list of words, check if the last one rhymes with any one of the rest



# loglik

In [ ]:
import kenlm
model = kenlm.LanguageModel('train.lm')
sent='in the beginning was the word'
loglik=model.score(sent)/len(sent)

# build training data

- for a line, extract features based on this line and previous line, label=1 
- for a line, extract features based on this line and a random line, label=0
- generate a large number of sents, for a low loglik generated line (bottom ranked K in the normalized-loglik list), extract features for this line and a random line from the corpus, label=0
- do we need to generate a lot of sentences for the negative examples?
- we could also impose some hard coded constraints on the loglik threshold.

In [96]:
import os
from spacy.en import English

#data_dir = os.environ.get('SPACY_DATA', LOCAL_DATA_DIR)

nlp = English()

doc3 = nlp(u"this is spacy lemmatize testing. programming books are more better than others")

print [token.lemma_ for token in doc3]
    

[u'this', u'be', u'spacy', u'lemmatize', u'testing', u'.', u'programming', u'book', u'be', u'more', u'better', u'than', u'others']


In [106]:
doc3=nlp(u"Like I'm a broken relates No he saves the jams")
print [token.lemma_ for token in doc3]

[u'like', u'i', u'be', u'a', u'broken', u'relate', u'no', u'he', u'save', u'the', u'jam']


In [107]:
words=u"Like I'm a broken relates No he saves the jams"
words=nlp(words)
words=[token.lemma_ for token in words]

In [108]:
words

[u'like',
 u'i',
 u'be',
 u'a',
 u'broken',
 u'relate',
 u'no',
 u'he',
 u'save',
 u'the',
 u'jam']

In [104]:

nlp(unicode("Hello world"))

Hello world

In [112]:
doc3 = nlp(u"Like I'm a broken relates")

for token in doc3:
    print token.lemma_

like
i
be
a
break
relate


In [111]:
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
doc3="relates"
for token in doc3.split(' '):
    print lmtzr.lemmatize(token)

relates
